In [1]:
import ast
import astunparse
import csv
import torch
import numpy as np
from pybase64 import b64decode
from typing import List
from collections import defaultdict
from IPython.display import clear_output
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
class Runner:
    def __init__(self, detector) -> None:
        self.detector = detector
        self.task_ids = []
        self.user_codes = []
        self.hints = []
        self.counter_task_solutions = defaultdict(lambda: 0)
        self.counter_task_errors = defaultdict(lambda: 0)

    def run(self):
        task_order = 0
        task_ids = []
        with open('./solutions-ipython.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=';')
            next(csv_reader)
    
            for row in csv_reader:
                task_id = row[1]
                solution_encoded = row[2]
                solution_decoded = b64decode(solution_encoded).decode("utf-8")
                
                self.counter_task_solutions[task_order] += 1
                self.detector.detect_and_show_hint(code=solution_decoded)

                if self.detector.detected:
                    self.counter_task_errors[task_order] += 1
                    self.task_ids.append(task_id)
                    self.user_codes.append(self.detector.user_code)
                    self.hints.append(self.detector.hint)

                if task_id not in task_ids:
                    task_ids.append(task_id)
                    task_order += 1

        print(f"Total number of detected errors: {len(self.hints)}")
        self.get_n_tasks_with_greater_than_10_errors()

    def increment_counter(self, counter, task_id):
        if task_id not in counter.keys():
            counter[task_id] = 1
        else:
            counter[task_id] += 1
    
    def display_detected_error(self, error_idx):
        print(
            "User code: \n",
            self.user_codes[error_idx],
        )
        print("\n------------------\n")
        print(
            "Hint: \n",
            self.hints[error_idx],
        )

    def get_n_tasks_with_greater_than_10_errors(self):
        self.n_tasks_with_greater_than_10_errors = 0
        for v in self.counter_task_errors.values():
            if v > 10:
                self.n_tasks_with_greater_than_10_errors += 1

In [3]:
class DuplicatedSegment:
    def __init__(self, lines_top: List[int], col_top: int, lines_bottom: List[int], col_bottom: int):
        self.lines_top = lines_top
        self.col_top = col_top
        self.lines_bottom = lines_bottom
        self.col_bottom = col_bottom
        self.lines = lines_top + lines_bottom

class DuplicatedCodeDetector:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("congcongwang/gpt2_medium_fine_tuned_coder")
        self.model = AutoModelForCausalLM.from_pretrained("congcongwang/gpt2_medium_fine_tuned_coder")     
        self.model.to("cuda:1")
        self.cosine_similarity = torch.nn.CosineSimilarity(dim=1)
        self.similarity_threshold = 0.97
        
    def _encode_subtrees(self):
        for node in self._nodes:
            code_str = astunparse.unparse(node)[:-1]
            input_ids = self.tokenizer.encode("<python> " + code_str, return_tensors='pt')
            n_tokens = input_ids.shape[-1]
            
            outputs = self.model.generate(
                input_ids=input_ids.to("cuda:1"),
                max_length=n_tokens + 1,
                num_return_sequences=1,
                output_hidden_states=True,
                return_dict_in_generate=True,
            )

            self._nodes_enc.append(outputs.hidden_states[0][-1][:, -1, :])
            clear_output()  
    
    def _get_child_nodes(self, node: ast.AST) -> None:
        self._nodes.extend(node.body)
        for n in ast.iter_child_nodes(node):
            if hasattr(n, "body"):
                self._get_child_nodes(n)

    def _get_duplicity_matrix_base(self, root_node: ast.AST) -> None:
        self._get_child_nodes(root_node)
        self._nodes.sort(key=lambda n: n.lineno)

    def _get_mask_same_type(self):
        n_nodes = len(self._nodes)
        self._mask_same_type = np.zeros((n_nodes, n_nodes), dtype=np.float32)
        
        for i in range(n_nodes):
            for j in range(n_nodes):
                if j < i:
                    self._mask_same_type[i, j] = self._mask_same_type[j, i]
                elif j == i:
                    self._mask_same_type[i, j] = 1.0
                else:
                    self._mask_same_type[i, j] = float(type(self._nodes[i]) == type(self._nodes[j]))

    def _get_mask_same_num_lines(self):
        n_nodes = len(self._nodes)
        self._mask_same_num_lines = np.zeros((n_nodes, n_nodes), dtype=np.float32)
        
        for i in range(n_nodes):
            for j in range(n_nodes):
                if j < i:
                    self._mask_same_num_lines[i, j] = self._mask_same_num_lines[j, i]
                elif j == i:
                    self._mask_same_num_lines[i, j] = 1.0
                else:
                    self._mask_same_num_lines[i, j] = len(self._get_node_lines_list(self._nodes[i])) == len(self._get_node_lines_list(self._nodes[j]))
                    

    def _compute_duplicity_matrix_entry(self, node_1, node_2):
        sim = self.cosine_similarity(node_1,node_2).cpu().numpy()[0]
        return sim
    
    def _compute_duplicity_matrix(self):
        n_nodes = len(self._nodes)
        self._duplicity_matrix = np.zeros((n_nodes, n_nodes), dtype=np.float32)
        for i in range(n_nodes):
            for j in range(n_nodes):
                if j < i:
                    self._duplicity_matrix[i, j] = self._duplicity_matrix[j, i]
                elif j == i:
                    self._duplicity_matrix[i, j] = 1.0
                else:
                    self._duplicity_matrix[i, j] = self._compute_duplicity_matrix_entry(
                        self._nodes_enc[i],
                        self._nodes_enc[j]
                    )

    def _scale_duplicity_matrix(self):
        min_similarity = np.amin(self._duplicity_matrix)
        self._duplicity_matrix = (self._duplicity_matrix - min_similarity) / (1 - min_similarity)

    def _apply_mask_same_type(self):
        self._duplicity_matrix = np.multiply(self._duplicity_matrix, self._mask_same_type)

    def _apply_mask_same_num_lines(self):
        self._duplicity_matrix = np.multiply(self._duplicity_matrix, self._mask_same_num_lines)

    def _apply_similarity_threshold(self):
        self._duplicity_matrix = (self._duplicity_matrix > self.similarity_threshold).astype(float)

    def _check_lines_already_highlighted(self, lines):
        higlighted_lines = []
        already_highlighted = False

        for s in self._duplicated_code_segments:
            higlighted_lines.extend(s.lines)

        for l in lines:
            if l in higlighted_lines:
                already_highlighted = True
                break
        
        return already_highlighted
    
    def _get_node_lines_list(self, node):
        return list(range(node.lineno, node.end_lineno + 1))
    
    def _get_duplicated_code_segments(self):
        n_nodes = len(self._nodes)

        for i in range(n_nodes):
            for j in range(i+1, n_nodes):
                if (self._duplicity_matrix[i][j] == 1.0
                    and not self._check_lines_already_highlighted(self._get_node_lines_list(self._nodes[j]))):
                    self._duplicated_code_segments.append(
                        DuplicatedSegment(
                            lines_top=self._get_node_lines_list(self._nodes[i]),
                            col_top=self._nodes[i].col_offset, 
                            lines_bottom=self._get_node_lines_list(self._nodes[j]),
                            col_bottom=self._nodes[j].col_offset,
                        )
                    )

    def _merge_neighbouring_code_segments(self):
        n_segments = len(self._duplicated_code_segments)
        for i in range(n_segments):
            for j in range(i+1, n_segments):
                if (self._duplicated_code_segments[i].lines_top[-1] + 1 == self._duplicated_code_segments[j].lines_top[0]
                    and self._duplicated_code_segments[i].lines_bottom[-1] + 1 == self._duplicated_code_segments[j].lines_bottom[0]
                    and self._duplicated_code_segments[i].col_top == self._duplicated_code_segments[j].col_top
                    and self._duplicated_code_segments[i].col_bottom == self._duplicated_code_segments[j].col_bottom):

                    self._duplicated_code_segments[i].lines_top += self._duplicated_code_segments[j].lines_top
                    self._duplicated_code_segments[i].lines_bottom += self._duplicated_code_segments[j].lines_bottom
                    self._duplicated_code_segments[i].lines += self._duplicated_code_segments[j].lines 

    def _remove_segment_idxs(self, segment_idxs_to_remove):
        n_segments = len(self._duplicated_code_segments)
        duplicated_code_segments = []
        for i in range(n_segments):
            if i not in segment_idxs_to_remove:
                duplicated_code_segments.append(self._duplicated_code_segments[i])

        self._duplicated_code_segments = duplicated_code_segments
    
    def _remove_overlapping_code_segments_top_top(self):
        n_segments = len(self._duplicated_code_segments)
        segment_idxs_to_remove = []
        for i in range(n_segments):
            for j in range(n_segments):
                if i != j:
                    n_lines_i = len(self._duplicated_code_segments[i].lines_top)
                    n_lines_j = len(self._duplicated_code_segments[j].lines_top)
                    smaller_segment_idx = i if n_lines_i < n_lines_j else j
                    if len(set(self._duplicated_code_segments[i].lines_top) & set(self._duplicated_code_segments[j].lines_top)) > 0:
                        segment_idxs_to_remove.append(smaller_segment_idx)

        self._remove_segment_idxs(segment_idxs_to_remove)

    def _remove_overlapping_code_segments_top_bottom(self):
        n_segments = len(self._duplicated_code_segments)
        segment_idxs_to_remove = []
        for i in range(n_segments):
            if len(set(self._duplicated_code_segments[i].lines_top) & set(self._duplicated_code_segments[i].lines_bottom)) > 0:
                segment_idxs_to_remove.append(i)

        self._remove_segment_idxs(segment_idxs_to_remove)

    def _remove_too_short_code_segments(self):
        n_segments = len(self._duplicated_code_segments)
        segment_idxs_to_remove = []
        for i in range(n_segments):
            if len(self._duplicated_code_segments[i].lines_top) < 2:
                segment_idxs_to_remove.append(i)

        self._remove_segment_idxs(segment_idxs_to_remove)

    def _compose_hint(self, code):
        code_strings = code.split('\n')
        self.hint: List[str] = []
        for lineno, code_string in enumerate(code_strings):
            string_is_part_of_duplicated_segment = False
            segment_idx = None
            for s_idx in range(len(self._duplicated_code_segments)):
                if lineno + 1 in self._duplicated_code_segments[s_idx].lines:
                    string_is_part_of_duplicated_segment = True
                    segment_idx = s_idx
    
            if string_is_part_of_duplicated_segment:
                code_string += f"               : {segment_idx}"
    
            self.hint.append(code_string)

        self.hint: str = "\n".join(self.hint)
    
    def _check_detected(self):
        self.detected = len(self._duplicated_code_segments) > 0
    
    def detect_and_show_hint(self, code: str) -> None:
        self.user_code = None
        self.hint = None
        self.detected = False

        self._nodes: List[ast.AST] = []
        self._nodes_enc: List[torch.Tensor] = []
        self._duplicity_matrix: np.ndarray = None
        self._duplicated_code_segments: List[DuplicatedSegment] = []
        self._mask_same_type: np.ndarray = None
        self._mask_same_num_lines: np.ndarray = None

        try:
            parsed = ast.parse(code)
        
            self._get_duplicity_matrix_base(parsed)
            self._get_mask_same_type()
            self._get_mask_same_num_lines()
            self._encode_subtrees()
            self._compute_duplicity_matrix()
            self._scale_duplicity_matrix()
            self._apply_mask_same_type()
            self._apply_mask_same_num_lines()
            self._apply_similarity_threshold()
            self._get_duplicated_code_segments()
            self._merge_neighbouring_code_segments()
            self._remove_overlapping_code_segments_top_top()
            self._remove_overlapping_code_segments_top_bottom()
            self._remove_too_short_code_segments()
            self._check_detected()
                
            if self.detected:
                self.user_code = code
                self._compose_hint(code)

        except Exception:
            self.user_code = None
            self.hint = None
            self.detected = False

            # ignore solutions where the code can not be executed (due to the user indentation errors - tabs, incorrect spacing) 
            return

In [4]:
runner_duplicated_code_detector = Runner(detector=DuplicatedCodeDetector())
runner_duplicated_code_detector.run()

Token indices sequence length is longer than the specified maximum sequence length for this model (1951 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Total number of detected errors: 365
